# Stoneburner, Kurt
- ## DSC 550 - Week 09/10

In [1]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)


### 2. Neural Network Classifier with Keras###
Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using Keras. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.

In [3]:
#https://oindrilasen.com/2021/02/how-to-install-and-import-keras-in-anaconda-jupyter-notebooks/

from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, confusion_matrix
import tensorflow as tf
import time

In [4]:
df = pd.read_pickle("z_wk09_categorized_comments_processed.zip")

df['processed'] = df['lema_stem_tokens'].apply(lambda word_list: ' '.join(word_list)) 

#//*** Convert categorical string to categorical int
#//*** Only run once to prevent iPython issues
if (df.dtypes['cat'] == object):
    cat_dict = dict(tuple(enumerate(df['cat'].unique())))
    #//*** Build sexcat Categorical column
    df['intcat'] = df['cat'].copy()
    
    #//*** replace values using the sex_dict dictionary
    for key,value in cat_dict.items():
        df['intcat'] = df['intcat'].replace(value,key)

In [149]:
#//*** Reference Code: Applied Text Analysys with Python p282.
metric_dict = {}

metrics = ['total_features',]

def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

def build_network():
    """
    Create a function that returns a compiled neural network
    """
    nn = Sequential()
    nn.add(Dense(N_FEATURES, activation='relu', input_shape=(N_FEATURES,)))
    
    nn.add(Dense(N_FEATURES*SECOND_LAYER_SIZE, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return nn

score_df = pd.DataFrame()


#//*** Theoretically, this number should be return as the second value of the tfidf sparse matrix
N_FEATURES = 1000

#//*** X is Post Processed Data to evaluate
data_model_x = df['processed']

tfidf = TfidfVectorizer(max_features=N_FEATURES)

data_model_y = df['intcat']

test_size=.5
test_size=.98
#continuous scoring model
#scoring = 'r2_score'

#data_model_x = tfidf.fit_transform(data_model_x)

#//*** N_CLASSES the number of categories to solve for
N_CLASSES = len(df['intcat'].unique())

FEATURE_PERCENT = .25
SECOND_LAYER_SIZE = .25
epochs = 2

for test_size in [.995,.98,.95,.9,.85,.8]:
    # split the data randomly into test/train sets
    x_train, x_test, y_train, y_test = train_test_split(data_model_x, data_model_y, test_size =test_size, random_state=0)


    #vectorizer = TfidfVectorizer(max_features=N_FEATURES)

    #//*** Initialize the Vectorizer, get all the features
    tfidf = TfidfVectorizer()

    print("First pass Vectorizing")
    total_features = tfidf.fit_transform(x_train).shape[1]


    for FEATURE_PERCENT in [.25,.5,.75,1]:
        # split the data randomly into test/train sets
        x_train, x_test, y_train, y_test = train_test_split(data_model_x, data_model_y, test_size =test_size, random_state=0)

        N_FEATURES = int(total_features * FEATURE_PERCENT)

        print(f"Re-Vectorizing: max_features={N_FEATURES} [{total_features}*{FEATURE_PERCENT}]")

        vectorizer = TfidfVectorizer(max_features=N_FEATURES)


        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)

        x_train = convert_sparse_matrix_to_sparse_tensor(x_train)
        x_test = convert_sparse_matrix_to_sparse_tensor(x_test)

        for SECOND_LAYER_SIZE in [.25,.5,.75,1]:

            for epochs in [5,10,20]:

                pipeline = Pipeline([
                #  ('norm', TextNormalizer()),
                #  ('vect', TfidfVectorizer(max_features=N_FEATURES)),
                 ('nn', KerasClassifier(build_fn=build_network,
                 epochs=epochs,
                batch_size=128))
                 ])

                start_time = time.time()

                pipeline.fit(x_train,y_train)

                model_run_time = time.time() - start_time 
                #print(f"Model Run Time: {model_run_time}s")
                precision = 1
                accuracy = 1
                recall = 1

                print('Scoring...')
                start_score_time = time.time()
                y_predicted = pipeline.predict(x_test)
                print(f"Scoring Time {round(time.time()-start_score_time,2)}s")

                precision = precision_score(y_test, y_predicted, average='micro')
                accuracy = accuracy_score(y_test, y_predicted)
                recall = recall_score(y_test, y_predicted, average ='micro')
                cm = confusion_matrix(y_test, y_predicted)

                print("========")
                print(f"Precision: {precision}")
                print(f"Accuracy: {accuracy}")
                print(f"Recall: {recall}")
                print(f"cm: {cm}")

                if len(score_df) == 0:
                    score_df = pd.DataFrame()
                    score_df['acc_per_s'] = [accuracy/model_run_time]    
                    score_df['time'] = [round(model_run_time,0)]  
                    score_df['size'] = [int((1-test_size)*len(data_model_x))]
                    score_df['tf'] = [total_features]    
                    score_df['fp'] = [FEATURE_PERCENT]    
                    score_df['features'] = [N_FEATURES]    
                    score_df['sls'] = [SECOND_LAYER_SIZE]    
                    score_df['epochs'] = [epochs]    
                    score_df['accuracy'] = [accuracy]    
                    score_df['precision'] = [precision]    
                    score_df['recall'] = [recall]    

                else:
                    temp_df = pd.DataFrame()
                    temp_df['acc_per_s'] = [accuracy/model_run_time]    
                    temp_df['time'] = [round(model_run_time,0)]    
                    temp_df['size'] = [int((1-test_size)*len(data_model_x))]
                    temp_df['tf'] = [total_features]    
                    temp_df['fp'] = [FEATURE_PERCENT]    
                    temp_df['features'] = [N_FEATURES]    
                    temp_df['sls'] = [SECOND_LAYER_SIZE]    
                    temp_df['epochs'] = [epochs]    
                    temp_df['accuracy'] = [accuracy]    
                    temp_df['precision'] = [precision]    
                    temp_df['recall'] = [recall] 

                    score_df = pd.concat([score_df,temp_df])
                print(temp_df)
    

print(score_df)

First pass Vectorizing
Re-Vectorizing: max_features=1802 [7210*0.25]
Epoch 1/5
24/24 [==============================] - 0s 16ms/step - loss: 0.7596 - accuracy: 0.6997
Epoch 2/5
24/24 [==============================] - 0s 16ms/step - loss: 0.5439 - accuracy: 0.7307
Epoch 3/5
24/24 [==============================] - 0s 16ms/step - loss: 0.3629 - accuracy: 0.8518
Epoch 4/5
24/24 [==============================] - 0s 15ms/step - loss: 0.2280 - accuracy: 0.9007
Epoch 5/5
24/24 [==============================] - 0s 16ms/step - loss: 0.1673 - accuracy: 0.9172
Scoring...
Scoring Time 15.1s
Precision: 0.7485812086013088
Accuracy: 0.7485812086013088
Recall: 0.7485812086013088
cm: [[ 48861      1  96100]
 [  2009     36  22925]
 [ 30562      3 402481]]
   acc_per_s  time  size    tf   fp  features  sls  epochs  accuracy  \
0   0.062956  12.0  3030  7210  0.5      3605    1       5  0.758953   

   precision    recall  
0   0.758953  0.758953  
Epoch 1/10
24/24 [==============================] - 0

24/24 [==============================] - 1s 27ms/step - loss: 0.7468 - accuracy: 0.6980
Epoch 2/20
24/24 [==============================] - 1s 27ms/step - loss: 0.5142 - accuracy: 0.7650
Epoch 3/20
24/24 [==============================] - 1s 27ms/step - loss: 0.2992 - accuracy: 0.8680
Epoch 4/20
24/24 [==============================] - 1s 27ms/step - loss: 0.1928 - accuracy: 0.9102
Epoch 5/20
24/24 [==============================] - 1s 29ms/step - loss: 0.1500 - accuracy: 0.9290
Epoch 6/20
24/24 [==============================] - 1s 28ms/step - loss: 0.1222 - accuracy: 0.9568
Epoch 7/20
24/24 [==============================] - 1s 28ms/step - loss: 0.1066 - accuracy: 0.9551
Epoch 8/20
24/24 [==============================] - 1s 28ms/step - loss: 0.1001 - accuracy: 0.9591
Epoch 9/20
24/24 [==============================] - 1s 28ms/step - loss: 0.0974 - accuracy: 0.9574
Epoch 10/20
24/24 [==============================] - 1s 28ms/step - loss: 0.0952 - accuracy: 0.9614
Epoch 11/20
24/24 [=

24/24 [==============================] - 2s 69ms/step - loss: 0.1910 - accuracy: 0.9053
Epoch 4/10
24/24 [==============================] - 2s 72ms/step - loss: 0.1363 - accuracy: 0.9277
Epoch 5/10
24/24 [==============================] - 2s 71ms/step - loss: 0.1068 - accuracy: 0.9653
Epoch 6/10
24/24 [==============================] - 2s 69ms/step - loss: 0.0809 - accuracy: 0.9729
Epoch 7/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0756 - accuracy: 0.9693
Epoch 8/10
24/24 [==============================] - 2s 68ms/step - loss: 0.0762 - accuracy: 0.9713
Epoch 9/10
24/24 [==============================] - 2s 69ms/step - loss: 0.0707 - accuracy: 0.9713
Epoch 10/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0739 - accuracy: 0.9723
Scoring...
Scoring Time 57.38s
Precision: 0.7603146383450142
Accuracy: 0.7603146383450142
Recall: 0.7603146383450142
cm: [[ 54329    271  90362]
 [  1723   3342  19905]
 [ 30879   1385 400782]]
   acc_per_s  time  size

24/24 [==============================] - 2s 93ms/step - loss: 0.7250 - accuracy: 0.7099
Epoch 2/5
24/24 [==============================] - 2s 97ms/step - loss: 0.3738 - accuracy: 0.8317
Epoch 3/5
24/24 [==============================] - 2s 95ms/step - loss: 0.1864 - accuracy: 0.9046
Epoch 4/5
24/24 [==============================] - 2s 95ms/step - loss: 0.1264 - accuracy: 0.9442
Epoch 5/5
24/24 [==============================] - 2s 95ms/step - loss: 0.0890 - accuracy: 0.9677
Scoring...
Scoring Time 94.46s
Precision: 0.7622599829512852
Accuracy: 0.7622599829512852
Recall: 0.7622599829512852
cm: [[ 52544    287  92131]
 [  1471   3693  19806]
 [ 28186   1471 403389]]
   acc_per_s  time  size    tf   fp  features  sls  epochs  accuracy  \
0   0.062013  12.0  3030  7210  0.5      3605    1       5   0.76226   

   precision   recall  
0    0.76226  0.76226  
Epoch 1/10
24/24 [==============================] - 2s 95ms/step - loss: 0.7252 - accuracy: 0.7040
Epoch 2/10
24/24 [================

Epoch 13/20
24/24 [==============================] - 3s 116ms/step - loss: 0.0584 - accuracy: 0.9769
Epoch 14/20
24/24 [==============================] - 3s 115ms/step - loss: 0.0581 - accuracy: 0.9766
Epoch 15/20
24/24 [==============================] - 3s 116ms/step - loss: 0.0572 - accuracy: 0.9772
Epoch 16/20
24/24 [==============================] - 3s 116ms/step - loss: 0.0568 - accuracy: 0.9772
Epoch 17/20
24/24 [==============================] - 3s 115ms/step - loss: 0.0577 - accuracy: 0.9762
Epoch 18/20
24/24 [==============================] - 3s 116ms/step - loss: 0.0587 - accuracy: 0.9766
Epoch 19/20
24/24 [==============================] - 3s 116ms/step - loss: 0.0559 - accuracy: 0.9762
Epoch 20/20
24/24 [==============================] - 3s 116ms/step - loss: 0.0563 - accuracy: 0.9772
Scoring...
Scoring Time 73.75s
Precision: 0.753642753135272
Accuracy: 0.753642753135272
Recall: 0.753642753135272
cm: [[ 69523    584  74855]
 [  2679   4729  17562]
 [ 49887   2981 380178]]
 

...

Epoch 13/20
24/24 [==============================] - 9s 391ms/step - loss: 0.0528 - accuracy: 0.9792
Epoch 14/20
24/24 [==============================] - 9s 391ms/step - loss: 0.0523 - accuracy: 0.9785
Epoch 15/20
24/24 [==============================] - 9s 393ms/step - loss: 0.0522 - accuracy: 0.9792
Epoch 16/20
24/24 [==============================] - 9s 392ms/step - loss: 0.0496 - accuracy: 0.9789
Epoch 17/20
24/24 [==============================] - 9s 392ms/step - loss: 0.0500 - accuracy: 0.9795
Epoch 18/20
24/24 [==============================] - 9s 392ms/step - loss: 0.0511 - accuracy: 0.9795
Epoch 19/20
24/24 [==============================] - 9s 391ms/step - loss: 0.0512 - accuracy: 0.9799
Epoch 20/20
24/24 [==============================] - 9s 394ms/step - loss: 0.0506 - accuracy: 0.9789
Scoring...
Scoring Time 387.23s
Precision: 0.7679815847344348
Accuracy: 0.7679815847344348
Recall: 0.7679815847344348
cm: [[ 52318    524  92120]
 [  1213   4700  19057]
 [ 24236   2752 406058

95/95 [==============================] - 7s 78ms/step - loss: 0.5968 - accuracy: 0.7559
Epoch 2/20
95/95 [==============================] - 8s 80ms/step - loss: 0.3204 - accuracy: 0.8710
Epoch 3/20
95/95 [==============================] - 8s 80ms/step - loss: 0.1593 - accuracy: 0.9428
Epoch 4/20
95/95 [==============================] - 8s 80ms/step - loss: 0.1074 - accuracy: 0.9606
Epoch 5/20
95/95 [==============================] - 8s 79ms/step - loss: 0.0918 - accuracy: 0.9655
Epoch 6/20
95/95 [==============================] - 8s 80ms/step - loss: 0.0834 - accuracy: 0.9664
Epoch 7/20
95/95 [==============================] - 8s 79ms/step - loss: 0.0789 - accuracy: 0.9677
Epoch 8/20
95/95 [==============================] - 8s 80ms/step - loss: 0.0751 - accuracy: 0.9680
Epoch 9/20
95/95 [==============================] - 8s 80ms/step - loss: 0.0746 - accuracy: 0.9678
Epoch 10/20
95/95 [==============================] - 8s 80ms/step - loss: 0.0727 - accuracy: 0.9686
Epoch 11/20
95/95 [=

Epoch 3/10
95/95 [==============================] - 11s 115ms/step - loss: 0.1510 - accuracy: 0.9464
Epoch 4/10
95/95 [==============================] - 11s 115ms/step - loss: 0.1047 - accuracy: 0.9611
Epoch 5/10
95/95 [==============================] - 11s 115ms/step - loss: 0.0879 - accuracy: 0.9657
Epoch 6/10
95/95 [==============================] - 11s 115ms/step - loss: 0.0800 - accuracy: 0.9661
Epoch 7/10
95/95 [==============================] - 11s 115ms/step - loss: 0.0754 - accuracy: 0.9684
Epoch 8/10
95/95 [==============================] - 11s 115ms/step - loss: 0.0740 - accuracy: 0.9684
Epoch 9/10
95/95 [==============================] - 11s 115ms/step - loss: 0.0716 - accuracy: 0.9691
Epoch 10/10
95/95 [==============================] - 11s 115ms/step - loss: 0.0707 - accuracy: 0.9696
Scoring...
Scoring Time 120.69s
Precision: 0.7899081981787812
Accuracy: 0.7899081981787812
Recall: 0.7899081981787812
cm: [[ 70072    496  72246]
 [  1637   6515  16428]
 [ 31697   2267 39253

Epoch 1/5
95/95 [==============================] - 29s 309ms/step - loss: 0.5877 - accuracy: 0.7563
Epoch 2/5
95/95 [==============================] - 29s 306ms/step - loss: 0.2576 - accuracy: 0.8993
Epoch 3/5
95/95 [==============================] - 29s 304ms/step - loss: 0.1215 - accuracy: 0.9569
Epoch 4/5
95/95 [==============================] - 29s 305ms/step - loss: 0.0826 - accuracy: 0.9682
Epoch 5/5
95/95 [==============================] - 29s 304ms/step - loss: 0.0725 - accuracy: 0.9719
Scoring...
Scoring Time 239.49s
Precision: 0.79785919230562
Accuracy: 0.79785919230562
Recall: 0.79785919230562
cm: [[ 77936    921  63957]
 [  1723   8763  14094]
 [ 35282   4072 387140]]
   acc_per_s   time   size     tf   fp  features  sls  epochs  accuracy  \
0    0.00541  147.0  12120  15096  0.5      7548  0.5       5  0.797859   

   precision    recall  
0   0.797859  0.797859  
Epoch 1/10
95/95 [==============================] - 30s 311ms/step - loss: 0.5905 - accuracy: 0.7569
Epoch 2/1

KeyboardInterrupt: 

Those permutations ran until my computer crashed.

I saved the results to a dataframe, which was corrupted somehow. Fortunately, I posted results in Teams, so I had a copy to post.

I had some interesting results in 9.2. I setup a number of loops to score models based on feature size and the size of the second layer.
size: 60600, represents a total sample size which is 10% of the data (using test_size = .9)
tf: total features across 60600 items
fp: Feature Percentage. Build the first layer with a percentage of the total features. Range [25,5,75,1]
features: The total number of features for the first layer - tf*fp = features
sls: Second Layer Size: Relative size of the second later compared to the first. Range [25,5,75,1]
acc_per_s: Accuracy / time. It's a metric to measure the cost of accuracy by time.
 
This sequence ran about 12 hours before crashing with an Out of Memory error when trying to create a first layer with 38639 features. The most interesting detail is how the accuracy didn't really change much by changing the hyperparameters. I'm thinking a smaller features size is probably just as effective as a larger one.

![title](z_wk09_9.2_results.png)